In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold # pa on SCC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix

import random
import math
from collections import Counter

In [2]:
df=pd.read_excel('../Data/summary_2ndEdition.xlsx', sheet_name='Sheet1', index_col='strain')  # ,index_col='strain',index_col='Mutant'
cs=pd.read_excel('../Data/summary_2ndEdition.xlsx', sheet_name='Sheet2')

print('df.shape',df.shape)
df = df.drop(df.query("strain == 'dummyVars'").index)
df=df[cs['carbon_source']]
df[df=='D']=1
df[df=='V']=1
df[df=='W']=1
df[df=='+']=2
df[df=='-']=0
df[df=='?']=None
df = df.dropna(axis=0, how='any')
print('df.shape',df.shape)
df = df[df.columns.sort_values()]
df.head()

df.shape (591, 91)
df.shape (462, 38)


,2-Keto-D-gluconate growth,"Butane 2,3 diol growth",Cellobiose growth,Citrate growth,D-Arabinose growth,D-Galactose growth,D-Galacturonate growth,D-Gluconate growth,"D-Glucono-1,5-lactone growth",D-Glucoronate growth,...,Methanol growth,"Propane 1,2 diol growth",Raffinose growth,Ribitol growth,Starch growth,Succinate growth,Sucrose growth,Xylitol growth,"a,a-Trehalose growth",myo-Inositol growth
strain,,,,,,,,,,,,,,,,,,,,,
Aciculoconidium aculeatum,2,0,2,2,0,0,0,0,0,0,...,0,0,0,1,1,2,2,0,2,0
Ambrosiozyma cicatricosa,2,0,2,2,0,0,0,0,1,0,...,0,0,0,2,0,2,2,2,2,0
Ambrosiozyma monospora,0,0,2,1,0,0,0,0,1,0,...,0,1,0,2,0,2,2,2,2,0
Arthroascus schoenii,0,0,0,0,0,0,2,0,0,0,...,0,1,0,0,1,2,1,0,1,0
Arxiozyma telluris,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
print(len(df.columns))
df.columns

38


Index(['2-Keto-D-gluconate growth', 'Butane 2,3 diol growth',
       'Cellobiose growth', 'Citrate growth', 'D-Arabinose growth',
       'D-Galactose growth', 'D-Galacturonate growth', 'D-Gluconate growth',
       'D-Glucono-1,5-lactone growth', 'D-Glucoronate growth',
       'D-Glucosamine growth', 'D-Glucose growth', 'D-Mannitol growth',
       'D-Ribose growth', 'D-Xylose growth', 'DL-Lactate growth',
       'Erythritol growth', 'Ethanol growth', 'Galactitol growth',
       'Glycerol growth', 'Inulin growth', 'L-Arabinitol growth',
       'L-Arabinose growth', 'L-Rhamnose growth', 'L-Sorbose growth',
       'Lactose growth', 'Maltose growth', 'Melibiose growth',
       'Methanol growth', 'Propane 1,2 diol growth', 'Raffinose growth',
       'Ribitol growth', 'Starch growth', 'Succinate growth', 'Sucrose growth',
       'Xylitol growth', 'a,a-Trehalose growth', 'myo-Inositol growth'],
      dtype='object')

In [4]:
df.std(axis=1).reset_index().sort_values(0, ascending=False)

,strain,0
273,Lipomyces japonicus,1.012019
136,Candida quercuum,1.007793
247,Hormoascus philentomus,1.007793
117,Candida nitratophila,0.996794
153,Candida solani,0.996794
...,...,...
272,Lipomyces anomalus,0.474079
293,Octosporomyces octosporus,0.474079
244,Hasegawaea japonica,0.4565
4,Arxiozyma telluris,0.311012


In [5]:
# remove invariable sample and 'Glucose' condition
df = df.loc[ df.std(axis=1)!=0, np.invert(df.columns.str.contains("Glucose")) ]

In [6]:
#df_new=df.replace(2,1)
X=df.values
n, d = X.shape
print((n, d))

(461, 37)


In [7]:
#X[X==0]=-1
X=X.astype('int64')
print(X.dtype)##dtype('int64')
X

int64


array([[2, 0, 2, ..., 0, 2, 0],
       [2, 0, 2, ..., 2, 2, 0],
       [0, 0, 2, ..., 2, 2, 0],
       ...,
       [1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       [2, 1, 0, ..., 1, 0, 2]])

## Random Forest classifier

In [8]:
df_z_all=pd.read_csv('../Results/MIP_classify_z_all76_greedy2.csv')
df_z_all=df_z_all.iloc[3:, range(2,df_z_all.shape[1],2)]
df_z_all=df_z_all[ sorted(df_z_all.columns) ]

#select_cols = []
#for c in df_z_all.columns:
#    if '_1.0' in c:
#        select_cols.append(c)
#select_cols.sort()
#df_z_all = df_z_all[select_cols]
#df_z_all

In [82]:
print(len(df_z_all.columns))
df_z_all.columns

37


,2-Keto-D-gluconate_1.0,"Butane 2,3 diol_1.0",Cellobiose_1.0,Citrate_1.0,D-Arabinose_1.0,D-Galactose_1.0,D-Galacturonate_1.0,D-Gluconate_1.0,"D-Glucono-1,5-lactone_1.0",D-Glucoronate_1.0,...,Methanol_1.0,"Propane 1,2 diol_1.0",Raffinose_1.0,Ribitol_1.0,Starch_1.0,Succinate_1.0,Sucrose_1.0,Xylitol_1.0,"a,a-Trehalose_1.0",myo-Inositol_1.0
3,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,0
4,1,1,1,1,1,1,1,1,1,0,...,0,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,0,...,0,1,1,1,1,1,0,1,1,1
6,1,1,1,1,1,1,1,1,1,0,...,0,1,1,1,1,1,0,1,1,1
7,1,1,1,1,1,1,0,1,1,0,...,0,1,1,1,1,1,0,1,1,1
8,1,0,1,1,1,1,0,1,1,0,...,0,1,1,1,1,1,0,1,1,1
9,1,0,1,1,1,1,0,1,1,0,...,0,1,1,1,1,1,0,1,1,0
10,1,0,1,1,1,1,0,1,1,0,...,0,1,1,1,1,1,0,1,1,0
11,1,0,1,1,1,1,0,1,1,0,...,0,1,1,1,1,1,0,1,1,0
12,1,0,1,1,1,1,0,1,1,0,...,0,1,1,1,1,1,0,1,1,0


In [10]:
df_z_all.shape

(34, 37)

In [11]:
names_raw=df.columns.values
n, d = X.shape  

z_last = df_z_all.iloc[-1,:].values
y_last = X[:,z_last==0] 


myseed = 2020
cvFolds= 3

In [12]:
def sample_attrs(attrs, r, max=100, seed = 1234567890):
    i = 0
    random.seed(seed)
    while i < max:
        preds = random.sample(attrs, r)
        preds.sort()
        resps = list(set(attrs) - set(preds))
        yield (preds, resps)
        i += 1

In [13]:
def entropy(data):
    l = len(data)
    if l <= 1:
        return 0
    counts = Counter()
    for d in data:
        counts[d] += 1
    ent = 0
    probs = [float(c) / len(data) for c in counts.values()]
    for p in probs:
        ent -= p * math.log(p, 2.)
    return ent

x_idxs_byentropy = df.apply(entropy, 0).reset_index().sort_values(0, ascending=False)
x_idxs_byentropy = list(x_idxs_byentropy.index)
print(df.columns[x_idxs_byentropy], len(x_idxs_byentropy))

Index(['D-Gluconate growth', 'Ribitol growth', 'Xylitol growth',
       'L-Sorbose growth', 'D-Ribose growth', 'DL-Lactate growth',
       'D-Galactose growth', 'L-Arabinose growth',
       'D-Glucono-1,5-lactone growth', '2-Keto-D-gluconate growth',
       'Citrate growth', 'D-Xylose growth', 'L-Arabinitol growth',
       'D-Arabinose growth', 'Cellobiose growth', 'a,a-Trehalose growth',
       'Propane 1,2 diol growth', 'Maltose growth', 'Glycerol growth',
       'Raffinose growth', 'D-Glucosamine growth', 'Sucrose growth',
       'L-Rhamnose growth', 'D-Mannitol growth', 'Ethanol growth',
       'Starch growth', 'Succinate growth', 'Erythritol growth',
       'D-Glucoronate growth', 'Lactose growth', 'Galactitol growth',
       'Butane 2,3 diol growth', 'D-Galacturonate growth', 'Melibiose growth',
       'myo-Inositol growth', 'Inulin growth', 'Methanol growth'],
      dtype='object') 37


In [115]:
stats_all=[]

attrs_pred_max = 100#300

attrs_idx = list(range(df_z_all.shape[1]))

#for r in range(1, df_z_all.shape[1]):
for r in [1,2,3,6,9,12,15,18,21,24,27,30,33,36]:

    # for cross-validation
    if r==df_z_all.shape[1]-1:
        folds = StratifiedKFold(n_splits=cvFolds, shuffle=True, random_state=myseed)
    else:
        folds = MultilabelStratifiedKFold(n_splits=cvFolds, shuffle=True, random_state=myseed)
    
    # MASS-selected attributes
    z=df_z_all.iloc[df_z_all.shape[0]-r,:].values
    x_idxs_MASS = [i for i, _ in enumerate(z) if _ == 1]
    y_idxs_MASS = [i for i, _ in enumerate(z) if _ == 0]

    # max-entropy-selected attributes
    x_idxs_ent = x_idxs_byentropy[:r]
    x_idxs_ent.sort()
    y_idxs_ent = list(set(attrs_idx) - set(x_idxs_ent))
    
    # draw random attributes
    if r == 1:
        attrs_remaining = list(set(attrs_idx) - set(x_idxs_MASS))
        idxs_rand = [([x], list(set(attrs_idx) - set([x]))) for x in attrs_remaining]
    else:
        idxs_rand = list(sample_attrs(attrs_idx, r, attrs_pred_max, myseed))

    # first list item is the MASS-selected attr set
    x_idxs = [x_idxs_MASS] + [x_idxs_ent] + [xx[0] for xx in idxs_rand]
    y_idxs = [y_idxs_MASS] + [y_idxs_ent] + [xx[1] for xx in idxs_rand]

    for i in range(len(x_idxs)):
        x_idx = x_idxs[i]
        y_idx = y_idxs[i]
        x_idx_short = '|'.join([str(x) for x in x_idx])

        y_split = X[:,y_idx]
        name_i = names_raw[y_idx]
    
        for fold_, (trn_idx, test_idx) in enumerate(folds.split(X,y_split)):        
            print('r, i, fold_, and x_idx==', r, i, fold_, x_idx_short, end=';  ')  
            train_X, test_X = X[trn_idx][:,x_idx], X[test_idx][:,x_idx]
            train_y, test_y = X[trn_idx][:,y_idx], X[test_idx][:,y_idx]
        
            if fold_==0:#,name_i
                print('train_y.shape, train_y/test_y.mean:',train_y.shape, round(train_y.mean(),2), round(test_y.mean(),2))
        
            y_te_pre = []
            for j in range(train_y.shape[1]):
                rfc=RandomForestClassifier(n_estimators=10, random_state=myseed,n_jobs=5,class_weight="balanced")
                rfc.fit(train_X, train_y[:,j])
                y_te_pre.append(rfc.predict(test_X))
            y_te_pre = np.array(y_te_pre).T
            tn_fp_fn_tp = confusion_matrix(test_y.ravel(), y_te_pre.ravel()).ravel()
            tn_fp_fn_tp = '|'.join( [str(x) for x in tn_fp_fn_tp] )

            stats_all.append([r, i, x_idx_short,fold_, tn_fp_fn_tp])

r, i, fold_, and x_idx== 1 0 0 14;  train_y.shape, train_y/test_y.mean: (307, 37) 0.86 0.85
r, i, fold_, and x_idx== 1 0 1 14;  r, i, fold_, and x_idx== 1 0 2 14;  r, i, fold_, and x_idx== 1 1 0 7;  train_y.shape, train_y/test_y.mean: (301, 37) 0.88 0.85
r, i, fold_, and x_idx== 1 1 1 7;  r, i, fold_, and x_idx== 1 1 2 7;  r, i, fold_, and x_idx== 1 2 0 0;  train_y.shape, train_y/test_y.mean: (306, 37) 0.86 0.84
r, i, fold_, and x_idx== 1 2 1 0;  r, i, fold_, and x_idx== 1 2 2 0;  r, i, fold_, and x_idx== 1 3 0 1;  train_y.shape, train_y/test_y.mean: (305, 37) 0.85 0.83
r, i, fold_, and x_idx== 1 3 1 1;  r, i, fold_, and x_idx== 1 3 2 1;  r, i, fold_, and x_idx== 1 4 0 2;  train_y.shape, train_y/test_y.mean: (310, 37) 0.87 0.88
r, i, fold_, and x_idx== 1 4 1 2;  r, i, fold_, and x_idx== 1 4 2 2;  r, i, fold_, and x_idx== 1 5 0 3;  train_y.shape, train_y/test_y.mean: (307, 37) 0.85 0.84
r, i, fold_, and x_idx== 1 5 1 3;  r, i, fold_, and x_idx== 1 5 2 3;  r, i, fold_, and x_idx== 1 6 0 

In [ ]:
# tn_fp_fn_tp = confusion_matrix(test_y.ravel()[:100],y_te_pre.ravel()[:100]).ravel()
# print( '|'.join( [str(x) for x in tn_fp_fn_tp] ) )

#tn, fp, fn, tp = tn_fp_fn_tp
#print( (tp+tn)/(tp+tn+fn+fp) )
#print( (tp+tn)/(tp+tn+fn+fp) )
#print( 0.5*((tp)/(tp+fn)+(tn)/(tn+fp)) )
#precision = tp/(tp+fp)
#recall = (tp)/(tp+fn)
#print( 2 * (precision * recall) / (precision + recall) )

In [116]:
# in the multiclass case the confusion matrix format is: true class (rows) vs. predicted class (columns)
#   the counts of the confuision matrix are ordered by (true class)_(predicted class)
score_results = pd.DataFrame.from_records(
    stats_all, columns=['# of predictors', 'i', 'predictors', 'fold', '0_0|0_1|0_2|1_0|1_1|1_2|2_0|2_1|2_2'])

output = f'../result/MIP_yeast_baseline_20230411_stats.csv'
score_results.to_csv(output, index=False)
score_results.tail()

,# of predictors,i,predictors,fold,0_0|0_1|0_2|1_0|1_1|1_2|2_0|2_1|2_2
4090,36,100,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18...,1,119|1|15|25|8|13|14|7|106
4091,36,100,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18...,2,124|5|6|15|18|11|13|6|110
4092,36,101,0|1|2|3|4|5|6|7|8|10|11|13|14|15|16|17|18|19|2...,0,13|11|3|7|34|13|3|26|198
4093,36,101,0|1|2|3|4|5|6|7|8|10|11|13|14|15|16|17|18|19|2...,1,13|10|5|9|39|21|6|16|189
4094,36,101,0|1|2|3|4|5|6|7|8|10|11|13|14|15|16|17|18|19|2...,2,16|9|2|9|29|23|1|13|206


In [14]:
print(test_y.ravel()[:30], y_te_pre.ravel()[:30] )
print(confusion_matrix(test_y.ravel()[:30], y_te_pre.ravel()[:30]))
df_z_all.columns[x_idxs[0]]

NameError: name 'test_y' is not defined

In [79]:
x_idxs[0]
df_z_all.columns[x_idxs[0]]

Index(['D-Gluconate_1.0', 'D-Xylose_1.0'], dtype='object')

In [83]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, jaccard_score, cohen_kappa_score, matthews_corrcoef

stats_all=[]
acc_all=[]
b_acc_all=[]
#f1_micro_all=[]
#f1_macro_all=[]
f1_weight_all=[]
jaccard_all=[]
#roc_auc_all=[]
cohen_kappa_all=[]
mcc_all=[]

attrs_pred_max = 300

attrs_idx = list(range(df_z_all.shape[1]))

for r in range(1, df_z_all.shape[1]-1): ## at r==36: ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multiclass-multioutput' instead.



    # for cross-validation
    if r==df_z_all.shape[1]-1:
        folds = StratifiedKFold(n_splits=cvFolds, shuffle=True, random_state=myseed)
    else:
        folds = MultilabelStratifiedKFold(n_splits=cvFolds, shuffle=True, random_state=myseed)
    
    # MASS-selected attributes
    z=df_z_all.iloc[df_z_all.shape[0]-r,:].values
    x_idxs_MASS = [i for i, _ in enumerate(z) if _ == 1]
    y_idxs_MASS = [i for i, _ in enumerate(z) if _ == 0]

    # max-entropy-selected attributes
    x_idxs_ent = x_idxs_byentropy[:r]
    x_idxs_ent.sort()
    y_idxs_ent = list(set(attrs_idx) - set(x_idxs_ent))
    
    # draw random attributes
    if r == 1:
        attrs_remaining = list(set(attrs_idx) - set(x_idxs_MASS))
        idxs_rand = [([x], list(set(attrs_idx) - set([x]))) for x in attrs_remaining]
    else:
        idxs_rand = list(sample_attrs(attrs_idx, r, attrs_pred_max, myseed))

    # first list item is the MASS-selected attr set
    x_idxs = [x_idxs_MASS] + [x_idxs_ent] + [xx[0] for xx in idxs_rand]
    y_idxs = [y_idxs_MASS] + [y_idxs_ent] + [xx[1] for xx in idxs_rand]

    for i in range(len(x_idxs)):
        x_idx = x_idxs[i]
        y_idx = y_idxs[i]
        x_idx_short = '|'.join([str(x) for x in x_idx])

        y_split = X[:,y_idx]
        name_i = names_raw[y_idx]
    
        for fold_, (trn_idx, test_idx) in enumerate(folds.split(X,y_split)):        
            print('r, i, fold_, and x_idx==', r, i, fold_, x_idx_short, end=';  ')  
            train_X, test_X = X[trn_idx][:,x_idx], X[test_idx][:,x_idx]
            train_y, test_y = X[trn_idx][:,y_idx], X[test_idx][:,y_idx]
        
            if fold_==0:#,name_i
                print('train_y.shape, train_y/test_y.mean:',train_y.shape, round(train_y.mean(),2), round(test_y.mean(),2))
        
            y_te_pre = []
            #y_te_pre_ps = np.empty((0,3))
            for j in range(train_y.shape[1]):
                rfc=RandomForestClassifier(n_estimators=10, random_state=myseed,n_jobs=7,class_weight="balanced")
                rfc.fit(train_X, train_y[:,j])
                y_te_pre.append(rfc.predict(test_X))
                #y_te_pre_ps = np.vstack( (y_te_pre_ps, rfc.predict_proba(test_X)) )
            y_te_pre = np.array(y_te_pre).T
            tn_fp_fn_tp = confusion_matrix(test_y.ravel(), y_te_pre.ravel()).ravel()
            tn_fp_fn_tp = '|'.join( [str(x) for x in tn_fp_fn_tp] )

            stats_all.append([r, i, x_idx_short,fold_, tn_fp_fn_tp])
            acc_all.append([r, i, x_idx_short,fold_, accuracy_score(test_y.ravel(), y_te_pre.ravel())])
            b_acc_all.append([r, i, x_idx_short,fold_, balanced_accuracy_score(test_y.ravel(), y_te_pre.ravel())])
            #f1_micro_all.append([r, i, x_idx_short,fold_, f1_score(test_y.ravel(), y_te_pre.ravel(), average='micro')])
            #f1_macro_all.append([r, i, x_idx_short,fold_, f1_score(test_y.ravel(), y_te_pre.ravel(), average='macro')])
            f1_weight_all.append([r, i, x_idx_short,fold_, f1_score(test_y.ravel(), y_te_pre.ravel(), average='weighted')])
            jaccard_all.append([r, i, x_idx_short,fold_, jaccard_score(test_y.ravel(), y_te_pre.ravel(), average='weighted')])
            #roc_auc_all.append([r, i, x_idx_short,fold_, roc_auc_score(test_y.ravel(), y_te_pre_ps, average='weighted', multi_class = 'ovo')])
            cohen_kappa_all.append([r, i, x_idx_short,fold_, cohen_kappa_score(test_y.ravel(), y_te_pre.ravel())])
            mcc_all.append([r, i, x_idx_short,fold_, matthews_corrcoef(test_y.ravel(), y_te_pre.ravel())])
            

r, i, fold_, and x_idx== 1 0 0 13;  train_y.shape, train_y/test_y.mean: (317, 36) 0.8 0.85
r, i, fold_, and x_idx== 1 0 1 13;  r, i, fold_, and x_idx== 1 0 2 13;  r, i, fold_, and x_idx== 1 1 0 7;  train_y.shape, train_y/test_y.mean: (313, 36) 0.81 0.85
r, i, fold_, and x_idx== 1 1 1 7;  r, i, fold_, and x_idx== 1 1 2 7;  r, i, fold_, and x_idx== 1 2 0 0;  train_y.shape, train_y/test_y.mean: (313, 36) 0.81 0.84
r, i, fold_, and x_idx== 1 2 1 0;  r, i, fold_, and x_idx== 1 2 2 0;  r, i, fold_, and x_idx== 1 3 0 1;  train_y.shape, train_y/test_y.mean: (315, 36) 0.83 0.87
r, i, fold_, and x_idx== 1 3 1 1;  r, i, fold_, and x_idx== 1 3 2 1;  r, i, fold_, and x_idx== 1 4 0 2;  train_y.shape, train_y/test_y.mean: (312, 36) 0.81 0.83
r, i, fold_, and x_idx== 1 4 1 2;  r, i, fold_, and x_idx== 1 4 2 2;  r, i, fold_, and x_idx== 1 5 0 3;  train_y.shape, train_y/test_y.mean: (311, 36) 0.81 0.83
r, i, fold_, and x_idx== 1 5 1 3;  r, i, fold_, and x_idx== 1 5 2 3;  r, i, fold_, and x_idx== 1 6 0 4

In [72]:
y_te_pre_ps.shape
rfc.predict_proba(test_X)
(train_X, train_y[:,j])
#np.vstack( (y_te_pre_ps, rfc.predict_proba(test_X)) )

(array([[0],
        [2],
        [2],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [0],
        [1],
        [2],
        [2],
        [1],
        [0],
        [2],
        [1],
        [2],
        [0],
        [2],
        [2],
        [0],
        [0],
        [2],
        [1],
        [2],
        [1],
        [2],
        [0],
        [0],
        [2],
        [2],
        [1],
        [2],
        [2],
        [2],
        [0],
        [0],
        [0],
        [1],
        [2],
        [2],
        [2],
        [0],
        [2],
        [1],
        [1],
        [0],
        [2],
        [0],
        [1],
        [0],
        [0],
        [2],
        [2],
        [2],
        [0],
        [2],
        [0],
        [2],
        [2],
        [2],
        [2],
        [2],
        [1],
        [1],
        [2],
        [2],
        [1],

In [84]:
pd.DataFrame.from_records(stats_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', '0_0|0_1|0_2|1_0|1_1|1_2|2_0|2_1|2_2']).to_csv(
    f'../Results/MIP_yeast_baseline_20230411_stats-stats.csv', index=False)
pd.DataFrame.from_records(acc_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'accuracy']).to_csv(
    f'../Results/MIP_yeast_baseline_20230411_stats-acc.csv', index=False)
pd.DataFrame.from_records(b_acc_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'balanced_accuracy']).to_csv(
    f'../Results/MIP_yeast_baseline_20230411_stats-bacc.csv', index=False)
pd.DataFrame.from_records(f1_weight_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'f1_score_weighted']).to_csv(
    f'../Results/MIP_yeast_baseline_20230411_stats-f1weighted.csv', index=False)
pd.DataFrame.from_records(jaccard_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'jaccard_score']).to_csv(
    f'../Results/MIP_yeast_baseline_20230411_stats-jaccard.csv', index=False)
pd.DataFrame.from_records(cohen_kappa_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'cohen_kappa_score']).to_csv(
    f'../Results/MIP_yeast_baseline_20230411_stats-cohen_kappa.csv', index=False)
pd.DataFrame.from_records(mcc_all,
                          columns=['# of predictors', 'i', 'predictors', 'fold', 'mcc']).to_csv(
    f'../Results/MIP_yeast_baseline_20230411_stats-mcc.csv', index=False)